In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models
from torchvision.utils import make_grid
import os
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [3]:
class Model(nn.Module):
    def __init__(self):
      super(Model, self).__init__()
      self.conv1 = nn.Conv2d(1, 16, 3)
      self.conv2 = nn.Conv2d(16, 32, 3)
      self.conv3 = nn.Conv2d(32, 64, 3)
      self.conv4 = nn.Conv2d(64, 32, 3)
      self.fc1 = nn.Linear(2048, 1024)
      self.fc2 = nn.Linear(1024, 128)
      self.fc3 = nn.Linear(128, 7)
      self.batch_norm_16 = nn.BatchNorm2d(16)
      self.batch_norm_32 = nn.BatchNorm2d(32)
      self.batch_norm_64 = nn.BatchNorm2d(64)
      self.drop_out = nn.Dropout2d(p=0.3)

    def forward(self, x):
      
      x = F.relu(self.batch_norm_16(self.conv1(x)))
      x = F.relu(self.drop_out(F.max_pool2d(self.batch_norm_32(self.conv2(x)), 2)))
      x = F.relu(self.drop_out(F.max_pool2d(self.batch_norm_64(self.conv3(x)), 2)))
      x = F.relu(self.drop_out(self.batch_norm_32(self.conv4(x))))
      x = x.view(-1, 2048)

      x = F.relu(self.drop_out(self.fc1(x)))
      x = F.relu(self.drop_out(self.fc2(x)))
      x = self.fc3(x)

      return x

model = Model()

In [4]:
model.load_state_dict(torch.load('/content/drive/MyDrive/model/model_rotate.pt'))
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([16, 1, 3, 3])
conv1.bias 	 torch.Size([16])
conv2.weight 	 torch.Size([32, 16, 3, 3])
conv2.bias 	 torch.Size([32])
conv3.weight 	 torch.Size([64, 32, 3, 3])
conv3.bias 	 torch.Size([64])
conv4.weight 	 torch.Size([32, 64, 3, 3])
conv4.bias 	 torch.Size([32])
fc1.weight 	 torch.Size([1024, 2048])
fc1.bias 	 torch.Size([1024])
fc2.weight 	 torch.Size([128, 1024])
fc2.bias 	 torch.Size([128])
fc3.weight 	 torch.Size([7, 128])
fc3.bias 	 torch.Size([7])
batch_norm_16.weight 	 torch.Size([16])
batch_norm_16.bias 	 torch.Size([16])
batch_norm_16.running_mean 	 torch.Size([16])
batch_norm_16.running_var 	 torch.Size([16])
batch_norm_16.num_batches_tracked 	 torch.Size([])
batch_norm_32.weight 	 torch.Size([32])
batch_norm_32.bias 	 torch.Size([32])
batch_norm_32.running_mean 	 torch.Size([32])
batch_norm_32.running_var 	 torch.Size([32])
batch_norm_32.num_batches_tracked 	 torch.Size([])
batch_norm_64.weight 	 torch.Size([64])
batch_norm_64.bias

In [5]:
model.eval()

Model(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=7, bias=True)
  (batch_norm_16): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_32): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_64): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_out): Dropout2d(p=0.3, inplace=False)
)

In [6]:
import cv2, requests

facecasc = cv2.CascadeClassifier("/content/drive/MyDrive/model/haarcascade_frontalface_default.xml")

def emotion_detect(url):
    # frame = cv2.imread('/content/drive/MyDrive/Colab Notebooks/facial_emotion_recog/오.png')
    raw = requests.get(url).content
    nparr = np.asarray(bytearray(raw), dtype=np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (255, 0, 0), 2)
        roi_gray = gray[y : y + h, x : x + w]
        cropped_img = np.expand_dims(
            np.expand_dims(cv2.resize(roi_gray, (16, 16)), -1), 0
        )
        cropped_img = torch.from_numpy(cropped_img)
        prediction = model(cropped_img)

        p = prediction.tolist()
        print(p)

        
        maxindex = int(np.argmax(prediction))
        
        print("DEBUG: {}".format(maxindex))
        text_img = Image.fromarray(frame)
        draw = ImageDraw.Draw(text_img)
        
        draw.text((x+5, y-30), emotion_dict[maxindex], font=font, fill=(255, 255, 255))
        
        frame = np.array(text_img)
        # cv2.putText(
        #     frame,
        #     emotion_dict[maxindex],
        #     (x + 20, y - 5),
        #     cv2.FONT_HERSHEY_SIMPLEX,
        #     1,
        #     (255, 255, 255),
        #     2,
        #     cv2.LINE_AA,
        # )
    try:
        print("result: {}".format(emotion_dict[maxindex]))
    except Exception as err:
        print(err)
        return

    return frame

In [38]:
url = input()
# out =requests.get(url).content
# print(out)
# emotion_detect(url)

raw = requests.get(url).content
nparr = np.asarray(bytearray(raw), dtype=np.uint8)
nparr = cv2.resize(nparr, (48, 48), 0)

nparr = nparr[np.newaxis, np.newaxis,:,:]

nparr = torch.from_numpy(nparr).float()

out = model(nparr) # predict
p = out.tolist()
print(p)
lis = out.flatten().tolist()
lis = list(map(lambda x: x+100, lis))
result = [int(round(a / sum(lis), 3)*100) for a in lis]
classes = ['neutral', 'fearful', 'sad', 'happy', 'surprised', 'angry', 'disgusted']

print("표정 :", classes)
print("확률 :",result)

maxindex = int(np.argmax(p))

print("결과 :",classes[maxindex])

https://imgnn.seoul.co.kr/img/upload/2015/05/07/SSI_20150507184218_V.jpg
[[-19.769227981567383, -88.49606323242188, 0.3338146507740021, 19.73162078857422, -1.4216477870941162, 16.407264709472656, -9.704778671264648]]
표정 : ['neutral', 'fearful', 'sad', 'happy', 'surprised', 'angry', 'disgusted']
확률 : [13, 1, 16, 19, 16, 18, 14]
결과 : happy
